In [53]:
import pandas as pd
import json
from datetime import datetime

# Cargar el archivo JSON
with open(r'C:\Users\sergi\PROYECTO POWER RANKING\esports-data\tournaments.json', 'r') as archivo:
    datos_torneos = json.load(archivo)

# Función para obtener la información de los partidos
def obtener_info_partidos(torneos):
    info_partidos = []

    for torneo in torneos:
        torneo_id = torneo['id']
        torneo_nombre = torneo['name']
        torneo_fecha_inicio = datetime.strptime(torneo['startDate'], '%Y-%m-%d')

        for stage in torneo['stages']:
            stage_name=stage['name']
            for section in stage['sections']:
                for partido in section['matches']:
                    partido_id = partido['id']
                    partido_tipo = partido['strategy']['type']
                    partido_count = partido['strategy']['count']
                    game1_ID = partido['games'][0]['id']
                    team1 = partido['teams'][0]['id']
                    if partido['teams'][0]['result'] is not None and 'outcome' in partido['teams'][0]['result']:
                        outcome1 = partido['teams'][0]['result']['outcome']
                    else:
                        outcome1 = None  # Otra acción adecuada en caso de que 'outcome' no exista o sea None
                    team2 = partido['teams'][1]['id']
                    if partido['teams'][1]['result'] is not None and 'outcome' in partido['teams'][1]['result']:
                        outcome2 = partido['teams'][1]['result']['outcome']
                    else:
                        outcome2 = None  # Otra acción adecuada en caso de que 'outcome' no exista o sea None

                    info_partidos.append({
                        'Torneo ID': torneo_id,
                        'Torneo Nombre': torneo_nombre,
                        'Fecha Torneo': torneo_fecha_inicio,
                        'Stage':stage_name,
                        'Partido ID': partido_id,
                        'Tipo': partido_tipo,
                        'Count': partido_count,
                        'Game1 ID': game1_ID,
                        'Team 1 ID': team1,
                        'Outcome 1': outcome1,
                        'Team 2 ID': team2,
                        'Outcome 2': outcome2
                    })

    return info_partidos

# Obtener la información de los partidos
info_partidos = obtener_info_partidos(datos_torneos)
df_tournaments = pd.DataFrame(info_partidos)


In [54]:
# Cargar el archivo JSON
with open(r"C:\Users\sergi\PROYECTO POWER RANKING\esports-data\leagues.json", 'r') as archivo:
    datos_leagues = json.load(archivo)

# Función para obtener la información de las ligas
def obtener_info_leagues(datos_leagues):
    info_leagues = []

    for league in datos_leagues:
        name = league['name']
        region = league['region']

        for tournament in league['tournaments']:
            tournament_id = tournament['id']

            info_leagues.append({
                'league_name': name,  # Cambiado de 'torneo_id' a 'name'
                'region': region,
                'tournament_id': tournament_id
            })

    return info_leagues

# Obtener la información de las ligas
info_leagues = obtener_info_leagues(datos_leagues)

# Crear un DataFrame a partir de la información de las ligas
df_leagues = pd.DataFrame(info_leagues)

In [55]:
with open(r"C:\Users\sergi\PROYECTO POWER RANKING\esports-data\teams.json", 'r') as archivo:
    datos_equipos = json.load(archivo)
df_teams = pd.DataFrame(datos_equipos)    
with open(r"C:\Users\sergi\PROYECTO POWER RANKING\esports-data\mapping_data.json", 'r') as archivo:
    datos_mapping = json.load(archivo)
df_mapping = pd.DataFrame(datos_mapping)
with open(r"C:\Users\sergi\PROYECTO POWER RANKING\games.json", 'r') as archivo:
    datos_games = json.load(archivo)
df_games = pd.DataFrame(datos_games)


In [56]:
df1 = df_tournaments.merge(df_mapping, left_on='Game1 ID', right_on='esportsGameId', how='left')

In [57]:
df1 = df1.dropna(subset=['platformGameId'])
df1 = df1.reset_index(drop=True)

In [58]:
df2 = df1.merge(df_teams, left_on='Team 1 ID', right_on='team_id', how='left')
df2 = df2.rename(columns={'name': 'Team1_Name'})

In [59]:
df3 = df2.merge(df_teams, left_on='Team 2 ID', right_on='team_id', how='left')
df3 = df3.rename(columns={'name': 'Team2_Name'})

In [60]:
df4 = df3.merge(df_games, left_on='platformGameId', right_on='platformGameId', how='left')

In [61]:
df5 = df4.merge(df_leagues, left_on='Torneo ID', right_on='tournament_id', how='left')

In [62]:
df6 = df5[['region', 'league_name', 'Torneo Nombre', 'Fecha Torneo', 'Stage', 'Team 1 ID','Team1_Name', 'Outcome 1', 'Team 2 ID','Team2_Name', 'Outcome 2', 'eventTime']]

In [63]:
# Definir una función personalizada para cambiar el formato de la columna
def cambiar_a_texto(numero):
    # Realizar la transformación deseada aquí
    nuevo_valor = str(numero)  # Convertir el número a una cadena de texto
    return nuevo_valor

# Crear una copia del DataFrame
df6_copy = df6.copy()

# Aplicar la función a la copia
df6_copy['Team 1 ID'] = df6_copy['Team 1 ID'].apply(cambiar_a_texto)
df6_copy['Team 2 ID'] = df6_copy['Team 2 ID'].apply(cambiar_a_texto)

# Asignar la copia de vuelta al DataFrame original
df6 = df6_copy

In [64]:
df6.to_csv('databaseV1.csv')